In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import math
import numpy as np
import plotly.express as px

In [2]:
#Importing the coffee import and export data published by the International Coffee Organization
exports = pd.read_csv(r'C:\Users\camwa\Downloads\Coffee_export.csv')
imports = pd.read_csv(r'C:\Users\camwa\Downloads\Coffee_import.csv')

In [3]:
#I am only interested in looking at the top 8 importers and exporters of coffee in the world. Therefore I am going to sort by the total exports and imports columns to locate the powerhouses
exports.head()

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,Total_export
0,Angola,5040000,4260000,4800000,2340000,480000,2460000,3120000,3000000,3240000,...,480000,480000,360000,540000,660000,660000,540000,540000,1380000,43320000
1,Bolivia (Plurinational State of),9360000,4440000,5760000,2820000,5040000,5640000,7380000,6660000,5820000,...,4440000,3780000,3300000,3720000,1800000,1320000,1560000,1320000,1200000,137460000
2,Brazil,1016160000,1270980000,1127460000,1070280000,1036380000,868080000,915060000,1008060000,1088640000,...,2028360000,1712940000,1899060000,-2147483648,-2147483648,2056140000,1855500000,2138220000,-2147483648,33807709056
3,Burundi,35100000,41280000,38760000,25080000,30480000,31680000,13440000,31740000,22440000,...,13080000,23520000,11700000,15120000,13800000,12240000,10140000,12120000,17580000,646200000
4,Cameroon,156660000,105120000,98760000,42300000,32760000,24420000,33840000,82080000,44760000,...,29400000,37320000,16320000,22500000,23400000,16860000,14700000,17220000,15000000,1399920000


In [22]:
#Brazil's 2014, 2015, and 2019 coffee export values need to be updated with more accurate data 

updated_values = {
    '2014': 37335.1728254,
    '2015': 37562.8467468,
    '2019': 40697.8637087
}

for year, value in updated_values.items():
    exports.loc[exports['Country'] == 'Brazil', year] = value
    
updated_years = ['2014', '2015', '2019']
for year in updated_years:
    exports.loc[exports['Country'] == 'Brazil', year] *= 60000

In [7]:
#I need to see the top 8 exporters by organzing the data with the 'Total_export' column
top_exporters = exports.sort_values(by='Total_export', ascending=False).head(8)

In [8]:
#Dropping the total_exports column
top_exporters.drop(top_exporters.columns[-1], axis=1, inplace=True)

In [11]:
# Now I am reconfiguring the data to be in long format instead of wide format
df_exports = pd.melt(top_exporters, id_vars='Country', var_name='Year', value_name='Value')
df_exports['Year'] = df_exports['Year'].astype(int)
df_exports

,Country,Year,Value
0,Brazil,1990,1.016160e+09
1,Viet Nam,1990,6.870000e+07
2,Colombia,1990,8.366400e+08
3,Indonesia,1990,4.141800e+08
4,India,1990,1.187400e+08
...,...,...,...
235,Indonesia,2019,3.800400e+08
236,India,2019,3.616800e+08
237,Guatemala,2019,2.167800e+08
238,Honduras,2019,4.059000e+08


In [12]:
#Adjusting the amount of coffee exported to be in the millions of kg
df_exports['Value'] = df_exports['Value'] / 1_000_000  
df_exports['Value'] = df_exports['Value'].round(2)
df_exports

,Country,Year,Value
0,Brazil,1990,1016.16
1,Viet Nam,1990,68.70
2,Colombia,1990,836.64
3,Indonesia,1990,414.18
4,India,1990,118.74
...,...,...,...
235,Indonesia,2019,380.04
236,India,2019,361.68
237,Guatemala,2019,216.78
238,Honduras,2019,405.90


In [24]:
#Now it's time to clean and transform the import data the way I did the export data 
top_importers = imports.sort_values(by='Total_import', ascending=False).head(8)
top_importers.drop(top_importers.columns[-1], axis=1, inplace=True)
df_imports = pd.melt(top_importers, id_vars='Country', var_name='Year', value_name='Value')
df_imports['Year'] = df_imports['Year'].astype(int)
df_imports['Value'] = df_imports['Value'] / 1_000_000  
df_imports['Value'] = df_imports['Value'].round(2)
df_imports['Country'].replace('United States of America', 'USA', inplace = True)
df_imports


,Country,Year,Value
0,USA,1990,1260.42
1,Germany,1990,820.26
2,Italy,1990,314.52
3,Japan,1990,319.80
4,France,1990,378.06
...,...,...,...
235,Japan,2019,482.40
236,France,2019,479.58
237,Spain,2019,359.04
238,United Kingdom,2019,333.24


In [28]:
# Time to create some interactive line plots with our data
exports_fig = px.line(df_exports, x='Year', y='Value', color='Country', title='Coffee Exports Over Time')
exports_fig.update_xaxes(title_text='Year',tickmode='linear', tickvals=df_exports['Year'].unique(),tickangle=45)
exports_fig.update_yaxes(title_text='Million Kgs of Coffee Exported')
exports_fig.update_traces(mode='markers+lines', hovertemplate='%{y:.2f}')
exports_fig.show()

In [30]:
imports_fig = px.line(df_imports, x='Year', y='Value', color='Country', title='Coffee Imports Over Time')
imports_fig.update_xaxes(title_text='Year',tickmode='linear', tickvals=df_imports['Year'].unique(),tickangle=45)
imports_fig.update_yaxes(title_text='Million Kgs of Coffee Imported')
imports_fig.update_traces(mode='markers+lines', hovertemplate='%{y:.2f}')
imports_fig.show()